In [ ]:
!pip install Levenshtein
!pip install gdown
!pip install Polygon3

In [ ]:
import gdown
import os

In [ ]:
url = 'https://drive.google.com/uc?id=1ibgN01mg4wFPMq0cwYFqp0h3gRnmD_EZ'
gdown.download(url, 'vintext_label.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ibgN01mg4wFPMq0cwYFqp0h3gRnmD_EZ
To: /content/vintext_label.zip
100%|██████████| 302k/302k [00:00<00:00, 11.4MB/s]


'vintext_label.zip'

In [ ]:
print(os.getcwd())

/content


# @converter

In [ ]:
dictionary = "aàáạảãâầấậẩẫăằắặẳẵAÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪeèéẹẻẽêềếệểễEÈÉẸẺẼÊỀẾỆỂỄoòóọỏõôồốộổỗơờớợởỡOÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠiìíịỉĩIÌÍỊỈĨuùúụủũưừứựửữƯỪỨỰỬỮUÙÚỤỦŨyỳýỵỷỹYỲÝỴỶỸ"


def make_groups():
    groups = []
    i = 0
    while i < len(dictionary) - 5:
        group = [c for c in dictionary[i : i + 6]]
        i += 6
        groups.append(group)
    return groups


groups = make_groups()

TONES = ["", "ˋ", "ˊ", "⸱", "ˀ", "˜"]
SOURCES = ["ă", "â", "Ă", "Â", "ê", "Ê", "ô", "ơ", "Ô", "Ơ", "ư", "Ư", "Đ", "đ"]
TARGETS = ["aˇ", "aˆ", "Aˇ", "Aˆ", "eˆ", "Eˆ", "oˆ", "o˒", "Oˆ", "O˒", "u˒", "U˒", "D^", "d^"]


def parse_tone(word):
    res = ""
    tone = ""
    for char in word:
        if char in dictionary:
            for group in groups:
                if char in group:
                    if tone == "":
                        tone = TONES[group.index(char)]
                    res += group[0]
        else:
            res += char
    res += tone
    return res


def full_parse(word):
    word = parse_tone(word)
    res = ""
    for char in word:
        if char in SOURCES:
            res += TARGETS[SOURCES.index(char)]
        else:
            res += char
    return res


def correct_tone_position(word):
    word = word[:-1]
    first_ord_char = ""
    second_order_char = ""
    for char in word:
        for group in groups:
            if char in group:
                second_order_char = first_ord_char
                first_ord_char = group[0]
    if len(word) >= 1 and word[-1] == first_ord_char and second_order_char != "":
        pair_chars = ["qu", "Qu", "qU", "QU", "gi", "Gi", "gI", "GI"]
        for pair in pair_chars:
            if pair in word and second_order_char in ["u", "U", "i", "I"]:
                return first_ord_char
        return second_order_char
    return first_ord_char


def decoder(recognition):
    for char in TARGETS:
        recognition = recognition.replace(char, SOURCES[TARGETS.index(char)])
    replace_char = correct_tone_position(recognition)
    if recognition[-1] in TONES:
        tone = recognition[-1]
        recognition = recognition[:-1]
        for group in groups:
            if replace_char in group:
                recognition = recognition.replace(replace_char, group[TONES.index(tone)])
    return recognition

# @ Eval

In [ ]:
CONFIDENT = 0.0

num_pts = 8

import os
import re
import shutil
import zipfile

source_dir = r"res_output"
if os.path.exists(source_dir):
    shutil.rmtree(source_dir)

!tar -xzf res_output_final6000.tar.gz
# !tar -xzf res_output_effortless.tar.gz

destination_dir = source_dir + "_filtered_thrs_" + str(CONFIDENT)

if os.path.exists(destination_dir):
    shutil.rmtree(destination_dir)
os.makedirs(destination_dir)

for filename in os.listdir(source_dir):
    if filename.endswith(".txt"):
        numbers = re.findall(r'\d+', filename)
        source_file = os.path.join(source_dir, filename)
        destination_file = os.path.join(destination_dir, '000' + str(int(numbers[0])) + '.txt')

        with open(source_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        filtered_lines = []
        # for line in lines:
        #     parts = line.strip().split(',', num_pts+1)
        #     if len(parts) == 10:
        #         coordinates = ','.join(parts[:num_pts])
        #         confidence = float(parts[num_pts])
        #         text = parts[num_pts+1]

        #         if confidence > CONFIDENT:
        #             filtered_line = f"{coordinates},####{decoder(text)}\n"
        #             # filtered_line = f"{coordinates},####{text}\n"
        #             filtered_lines.append(filtered_line)


        for line in lines:
            parts = line.strip().split(',', num_pts+1)
            if len(parts) == 10:
                coordinates = ','.join(parts[:num_pts])
                confidence = float(parts[num_pts][4:])
                text = parts[num_pts+1][4:]

                if confidence > CONFIDENT:
                    filtered_line = f"{coordinates},####{text}\n"
                    filtered_lines.append(filtered_line)

            with open(destination_file, 'w', encoding='utf-8') as file:
                file.writelines(filtered_lines)

shutil.make_archive(destination_dir, 'zip', destination_dir)

print("Đã hoàn thành việc sao chép và lọc các tệp.")
print(destination_dir + ".zip")

Đã hoàn thành việc sao chép và lọc các tệp.
res_output_filtered_thrs_0.0.zip


In [ ]:
import json
import os
import re
import zipfile
import numpy as np
import importlib
import sys
from collections import namedtuple
import Levenshtein as lstn
import codecs
from shapely.geometry import *


def evaluate_with_official_code(result_path, gt_path):
    _word_spotting = True
    return text_eval_main(
        det_file=result_path, gt_file=gt_path, is_word_spotting=_word_spotting
    )


def text_eval_main(det_file, gt_file, is_word_spotting):
    global WORD_SPOTTING
    WORD_SPOTTING = is_word_spotting
    return main_evaluation(
        None,
        det_file,
        gt_file,
        default_evaluation_params,
        validate_data,
        evaluate_method,
    )


def main_evaluation(
    p,
    det_file,
    gt_file,
    default_evaluation_params_fn,
    validate_data_fn,
    evaluate_method_fn,
    show_result=True,
    per_sample=True,
):
    """
    This process validates a method, evaluates it and if it succed generates a ZIP file with a JSON entry for each sample.
    Params:
    p: Dictionary of parmeters with the GT/submission locations. If None is passed, the parameters send by the system are used.
    default_evaluation_params_fn: points to a function that returns a dictionary with the default parameters used for the evaluation
    validate_data_fn: points to a method that validates the corrct format of the submission
    evaluate_method_fn: points to a function that evaluated the submission and return a Dictionary with the results
    """

    # if (p == None):
    #     p = dict([s[1:].split('=') for s in sys.argv[1:]])
    #     if(len(sys.argv)<3):
    #         print_help()
    p = {}
    p["g"] = gt_file  #'tttgt.zip'
    p["s"] = det_file  #'det.zip'

    evalParams = default_evaluation_params_fn()
    if "p" in p.keys():
        evalParams.update(
            p["p"] if isinstance(p["p"], dict) else json.loads(p["p"][1:-1])
        )

    resDict = {"calculated": True, "Message": "", "method": "{}", "per_sample": "{}"}
    # try:
    validate_data_fn(p["g"], p["s"], evalParams)
    evalData = evaluate_method_fn(p["g"], p["s"], evalParams)
    resDict.update(evalData)

    # except Exception as e:
    # resDict['Message']= str(e)
    # resDict['calculated']=False

    if "o" in p:
        if not os.path.exists(p["o"]):
            os.makedirs(p["o"])

        resultsOutputname = p["o"] + "/results.zip"
        outZip = zipfile.ZipFile(resultsOutputname, mode="w", allowZip64=True)

        del resDict["per_sample"]
        if "output_items" in resDict.keys():
            del resDict["output_items"]

        outZip.writestr("method.json", json.dumps(resDict))

    if not resDict["calculated"]:
        if show_result:
            sys.stderr.write("Error!\n" + resDict["Message"] + "\n\n")
        if "o" in p:
            outZip.close()
        return resDict

    if "o" in p:
        if per_sample == True:
            for k, v in evalData["per_sample"].items():
                outZip.writestr(k + ".json", json.dumps(v))

            if "output_items" in evalData.keys():
                for k, v in evalData["output_items"].items():
                    outZip.writestr(k, v)

        outZip.close()

    if show_result:
        sys.stdout.write("Calculated!")
        sys.stdout.write("\n")
        sys.stdout.write(json.dumps(resDict["e2e_method"]))
        sys.stdout.write("\n")
        sys.stdout.write(json.dumps(resDict["det_only_method"]))
        sys.stdout.write("\n")

    return resDict


def default_evaluation_params():
    """
    default_evaluation_params: Default parameters to use for the validation and evaluation.
    """
    global WORD_SPOTTING
    return {
        "IOU_CONSTRAINT": 0.5,
        "AREA_PRECISION_CONSTRAINT": 0.5,
        "WORD_SPOTTING": WORD_SPOTTING,
        "MIN_LENGTH_CARE_WORD": 0,
        "GT_SAMPLE_NAME_2_ID": "([0-9]+).txt",
        "DET_SAMPLE_NAME_2_ID": "([0-9]+).txt",
        "LTRB": False,  # LTRB:2points(left,top,right,bottom) or 4 points(x1,y1,x2,y2,x3,y3,x4,y4)
        "CRLF": False,  # Lines are delimited by Windows CRLF format
        "CONFIDENCES": False,  # Detections must include confidence value. MAP and MAR will be calculated,
        "SPECIAL_CHARACTERS": str("!?.:,*\"()·[]/'"),
        "ONLY_REMOVE_FIRST_LAST_CHARACTER": True,
    }


def validate_data(gtFilePath, submFilePath, evaluationParams):
    """
    Method validate_data: validates that all files in the results folder are correct (have the correct name contents).
                            Validates also that there are no missing files in the folder.
                            If some error detected, the method raises the error
    """
    gt = load_zip_file(gtFilePath, evaluationParams["GT_SAMPLE_NAME_2_ID"])

    subm = load_zip_file(submFilePath, evaluationParams["DET_SAMPLE_NAME_2_ID"], True)


def evaluation_imports():
    """
    evaluation_imports: Dictionary ( key = module name , value = alias  )  with python modules used in the evaluation.
    """
    return {"Polygon": "plg", "numpy": "np"}


def evaluate_method(gtFilePath, submFilePath, evaluationParams):
    """
    Method evaluate_method: evaluate method and returns the results
        Results. Dictionary with the following values:
        - method (required)  Global method metrics. Ex: { 'Precision':0.8,'Recall':0.9 }
        - samples (optional) Per sample metrics. Ex: {'sample1' : { 'Precision':0.8,'Recall':0.9 } , 'sample2' : { 'Precision':0.8,'Recall':0.9 }
    """
    for module, alias in evaluation_imports().items():
        globals()[alias] = importlib.import_module(module)

    def polygon_from_points(points):
        """
        Returns a Polygon object to use with the Polygon2 class from a list of 8 points: x1,y1,x2,y2,x3,y3,x4,y4
        """
        num_points = len(points)
        # resBoxes=np.empty([1,num_points],dtype='int32')
        resBoxes = np.empty([1, num_points], dtype="float32")
        for inp in range(0, num_points, 2):
            resBoxes[0, int(inp / 2)] = float(points[int(inp)])
            resBoxes[0, int(inp / 2 + num_points / 2)] = float(points[int(inp + 1)])
        pointMat = resBoxes[0].reshape([2, int(num_points / 2)]).T
        return plg.Polygon(pointMat)

    def rectangle_to_polygon(rect):
        resBoxes = np.empty([1, 8], dtype="int32")
        resBoxes[0, 0] = int(rect.xmin)
        resBoxes[0, 4] = int(rect.ymax)
        resBoxes[0, 1] = int(rect.xmin)
        resBoxes[0, 5] = int(rect.ymin)
        resBoxes[0, 2] = int(rect.xmax)
        resBoxes[0, 6] = int(rect.ymin)
        resBoxes[0, 3] = int(rect.xmax)
        resBoxes[0, 7] = int(rect.ymax)

        pointMat = resBoxes[0].reshape([2, 4]).T

        return plg.Polygon(pointMat)

    def rectangle_to_points(rect):
        points = [
            int(rect.xmin),
            int(rect.ymax),
            int(rect.xmax),
            int(rect.ymax),
            int(rect.xmax),
            int(rect.ymin),
            int(rect.xmin),
            int(rect.ymin),
        ]
        return points

    def get_union(pD, pG):
        areaA = pD.area()
        areaB = pG.area()
        return areaA + areaB - get_intersection(pD, pG)

    def get_intersection_over_union(pD, pG):
        # print("i: ", get_intersection(pD, pG))
        # print("u: ", get_union(pD, pG))
        try:
            return get_intersection(pD, pG) / get_union(pD, pG)
        except:
            return 0

    def get_intersection(pD, pG):
        # print("pD", pD)
        # print("pG", pG)
        pInt = pD & pG
        if len(pInt) == 0:
            return 0
        return pInt.area()

    def compute_ap(confList, matchList, numGtCare):
        correct = 0
        AP = 0
        if len(confList) > 0:
            confList = np.array(confList)
            matchList = np.array(matchList)
            sorted_ind = np.argsort(-confList)
            confList = confList[sorted_ind]
            matchList = matchList[sorted_ind]
            for n in range(len(confList)):
                match = matchList[n]
                if match:
                    correct += 1
                    AP += float(correct) / (n + 1)

            if numGtCare > 0:
                AP /= numGtCare

        return AP

    def transcription_match(
        transGt,
        transDet,
        specialCharacters=str(r'!?.:,*"()·[]/\''),
        onlyRemoveFirstLastCharacterGT=True,
    ):

        if onlyRemoveFirstLastCharacterGT:
            # special characters in GT are allowed only at initial or final position
            if transGt == transDet:
                return True

            if specialCharacters.find(transGt[0]) > -1:
                if transGt[1:] == transDet:
                    return True

            if specialCharacters.find(transGt[-1]) > -1:
                if transGt[0 : len(transGt) - 1] == transDet:
                    return True

            if (
                specialCharacters.find(transGt[0]) > -1
                and specialCharacters.find(transGt[-1]) > -1
            ):
                if transGt[1 : len(transGt) - 1] == transDet:
                    return True
            return False
        else:
            # Special characters are removed from the begining and the end of both Detection and GroundTruth
            while len(transGt) > 0 and specialCharacters.find(transGt[0]) > -1:
                transGt = transGt[1:]

            while len(transDet) > 0 and specialCharacters.find(transDet[0]) > -1:
                transDet = transDet[1:]

            while len(transGt) > 0 and specialCharacters.find(transGt[-1]) > -1:
                transGt = transGt[0 : len(transGt) - 1]

            while len(transDet) > 0 and specialCharacters.find(transDet[-1]) > -1:
                transDet = transDet[0 : len(transDet) - 1]

            return transGt == transDet

    def include_in_dictionary(transcription):
        """
        Function used in Word Spotting that finds if the Ground Truth transcription meets the rules to enter into the dictionary. If not, the transcription will be cared as don't care
        """
        # special case 's at final
        if (
            transcription[len(transcription) - 2 :] == "'s"
            or transcription[len(transcription) - 2 :] == "'S"
        ):
            transcription = transcription[0 : len(transcription) - 2]

        # hypens at init or final of the word
        transcription = transcription.strip("-")

        specialCharacters = str("'!?.:,*\"()·[]/")
        # specialCharacters = str("`")
        for character in specialCharacters:
            transcription = transcription.replace(character, " ")

        transcription = transcription.strip()

        if len(transcription) != len(transcription.replace(" ", "")):
            # print("len(transcription.replace(" ", "")): xx: ", transcription)
            return False

        # if len(transcription) < evaluationParams["MIN_LENGTH_CARE_WORD"]:
            # print("evaluati] xxxx: ", evaluationParams["MIN_LENGTH_CARE_WORD"])
            # print("evaluationParams xx: ", transcription)
            # return False

        notAllowed = str("×÷·")

        range1 = [ord("a"), ord("z")]
        range2 = [ord("A"), ord("Z")]
        range3 = [ord("À"), ord("ƿ")]
        range4 = [ord("Ǆ"), ord("ɿ")]
        range5 = [ord("Ά"), ord("Ͽ")]
        range6 = [ord("-"), ord("-")]
        range7 = [ord("Ạ"), ord("Ỹ")]
        range8 = [ord("0"), ord("9")]

        for char in transcription:
            charCode = ord(char)
            if notAllowed.find(char) != -1:
                #c print("notAllowed xx: ", transcription)
                return False

            valid = (
                (charCode >= range1[0] and charCode <= range1[1])
                or (charCode >= range2[0] and charCode <= range2[1])
                or (charCode >= range3[0] and charCode <= range3[1])
                or (charCode >= range4[0] and charCode <= range4[1])
                or (charCode >= range5[0] and charCode <= range5[1])
                or (charCode >= range6[0] and charCode <= range6[1])
                or (charCode >= range7[0] and charCode <= range7[1])
                or (charCode >= range8[0] and charCode <= range8[1])
            )
            if valid == False:
                #c print("transcription xx: ", transcription)
                return False

        # print("include_in_dictionary: ", transcription)
        # print("include_in_dictionary 2: ", valid)
        return True

    def include_in_dictionary_transcription(transcription):
        """
        Function applied to the Ground Truth transcriptions used in Word Spotting. It removes special characters or terminations
        """
        # special case 's at final
        if (
            transcription[len(transcription) - 2 :] == "'s"
            or transcription[len(transcription) - 2 :] == "'S"
        ):
            transcription = transcription[0 : len(transcription) - 2]

        # hypens at init or final of the word
        transcription = transcription.strip("-")

        specialCharacters = str("'!?.:,*\"()·[]/")\
        # specialCharacters = str("`")
        for character in specialCharacters:
            transcription = transcription.replace(character, " ")

        transcription = transcription.strip()

        return transcription

    perSampleMetrics = {}

    matchedSum = 0
    det_only_matchedSum = 0

    Rectangle = namedtuple("Rectangle", "xmin ymin xmax ymax")

    gt = load_zip_file(gtFilePath, evaluationParams["GT_SAMPLE_NAME_2_ID"])
    subm = load_zip_file(submFilePath, evaluationParams["DET_SAMPLE_NAME_2_ID"], True)
    print("submission file path: ", submFilePath)
    print("gt file path: ", gtFilePath)
    numGlobalCareGt = 0
    numGlobalCareDet = 0
    det_only_numGlobalCareGt = 0
    det_only_numGlobalCareDet = 0

    arrGlobalConfidences = []
    arrGlobalMatches = []

    x = 0
    y = 0
    max_fail = 0
    name_fail = ""
    for resFile in gt:
        # print(resFile)
        a = 0
        b = 0
        # print('resgt', resFile)
        gtFile = decode_utf8(gt[resFile])
        if gtFile is None:
            raise Exception("The file %s is not UTF-8" % resFile)

        recall = 0
        precision = 0
        hmean = 0
        detCorrect = 0
        detOnlyCorrect = 0
        iouMat = np.empty([1, 1])
        gtPols = []
        detPols = []
        gtTrans = []
        detTrans = []
        gtPolPoints = []
        detPolPoints = []
        gtDontCarePolsNum = (
            []
        )  # Array of Ground Truth Polygons' keys marked as don't Care
        det_only_gtDontCarePolsNum = []
        detDontCarePolsNum = (
            []
        )  # Array of Detected Polygons' matched with a don't Care GT
        det_only_detDontCarePolsNum = []
        detMatchedNums = []
        pairs = []

        arrSampleConfidences = []
        arrSampleMatch = []
        sampleAP = 0

        (
            pointsList,
            _,
            transcriptionsList,
        ) = get_tl_line_values_from_file_contents(
            gtFile, evaluationParams["CRLF"], evaluationParams["LTRB"], True, False
        )
        # print("pointsList a: ", len(pointsList))
        # print("transcriptionsList a: ", len(transcriptionsList))

        for n in range(len(pointsList)):
            points = pointsList[n]
            transcription = transcriptionsList[n]
            # print("transcriptionsList[n]: ", transcriptionsList[n])
            det_only_dontCare = dontCare = (
                transcription == "###"
            )  # ctw1500 and total_text gt have been modified to the same format.
            # print("dc 2: ", dontCare)
            if evaluationParams["LTRB"]:
                gtRect = Rectangle(*points)
                gtPol = rectangle_to_polygon(gtRect)
            else:
                gtPol = polygon_from_points(points)
            gtPols.append(gtPol)
            gtPolPoints.append(points)

            # On word spotting we will filter some transcriptions with special characters
            # print("dc 2.5: ", dontCare)
            if evaluationParams["WORD_SPOTTING"]:
                if dontCare == False:
                    if include_in_dictionary(transcription) == False:
                        # if(resFile == '0001541'):
                        #   print("dc 3: ", transcription)
                        dontCare = True
                    else:
                        transcription = include_in_dictionary_transcription(
                            transcription
                        )

            gtTrans.append(transcription)
            # print("dontCare 2.9: ", dontCare)
            if dontCare:
                gtDontCarePolsNum.append(len(gtPols) - 1)
                # print("dontCare 3: ", transcriptionsList[len(gtPols) - 1])
            if det_only_dontCare:
                # print("det_only_dontCare 3: ", transcriptionsList[len(gtPols) - 1])
                det_only_gtDontCarePolsNum.append(len(gtPols) - 1)

        if resFile in subm:

            detFile = decode_utf8(subm[resFile])

            (
                pointsList,
                confidencesList,
                transcriptionsList,
            ) = get_tl_line_values_from_file_contents_det(
                detFile,
                evaluationParams["CRLF"],
                evaluationParams["LTRB"],
                True,
                evaluationParams["CONFIDENCES"],
            )

            # print("transcriptionsList if resFile in subm:: ", transcriptionsList)
            for n in range(len(pointsList)):
                points = pointsList[n]
                transcription = transcriptionsList[n]
                # print("transcriptionsList[99]: ", transcriptionsList[n])

                if evaluationParams["LTRB"]:
                    detRect = Rectangle(*points)
                    detPol = rectangle_to_polygon(detRect)
                else:
                    detPol = polygon_from_points(points)
                detPols.append(detPol)
                detPolPoints.append(points)
                transcription = include_in_dictionary_transcription(transcription)
                detTrans.append(transcription)

                if len(gtDontCarePolsNum) > 0:
                    for dontCarePol in gtDontCarePolsNum:
                        i = dontCarePol
                        dontCarePol = gtPols[dontCarePol]
                        intersected_area = get_intersection(dontCarePol, detPol)
                        pdDimensions = detPol.area()
                        precision = (
                            0 if pdDimensions == 0 else intersected_area / pdDimensions
                        )
                        if precision > evaluationParams["AREA_PRECISION_CONSTRAINT"]:
                            # print("gtTrans: ", gtTrans[i])
                            # print("transcription bi trung: ", transcription)
                            detDontCarePolsNum.append(len(detPols) - 1)
                            break

                if len(det_only_gtDontCarePolsNum) > 0:
                    for dontCarePol in det_only_gtDontCarePolsNum:
                        dontCarePol = gtPols[dontCarePol]
                        intersected_area = get_intersection(dontCarePol, detPol)
                        pdDimensions = detPol.area()
                        precision = (
                            0 if pdDimensions == 0 else intersected_area / pdDimensions
                        )
                        if precision > evaluationParams["AREA_PRECISION_CONSTRAINT"]:
                            det_only_detDontCarePolsNum.append(len(detPols) - 1)
                            break

            if len(gtPols) > 0 and len(detPols) > 0:
                # Calculate IoU and precision matrixs
                outputShape = [len(gtPols), len(detPols)]
                iouMat = np.empty(outputShape)
                gtRectMat = np.zeros(len(gtPols), np.int8)
                detRectMat = np.zeros(len(detPols), np.int8)
                det_only_gtRectMat = np.zeros(len(gtPols), np.int8)
                det_only_detRectMat = np.zeros(len(detPols), np.int8)
                for gtNum in range(len(gtPols)):
                    for detNum in range(len(detPols)):
                        pG = gtPols[gtNum]
                        pD = detPols[detNum]
                        iouMat[gtNum, detNum] = get_intersection_over_union(pD, pG)

                        # print("iou: ", iouMat[gtNum, detNum])

                for gtNum in range(len(gtPols)):
                    for detNum in range(len(detPols)):
                        if (
                            gtRectMat[gtNum] == 0
                            and detRectMat[detNum] == 0
                            and gtNum not in gtDontCarePolsNum
                            and detNum not in detDontCarePolsNum
                        ):
                            if (
                                iouMat[gtNum, detNum]
                                > evaluationParams["IOU_CONSTRAINT"]
                            ):
                                gtRectMat[gtNum] = 1
                                detRectMat[detNum] = 1
                                # detection matched only if transcription is equal
                                # det_only_correct = True
                                # detOnlyCorrect += 1
                                if evaluationParams["WORD_SPOTTING"]:
                                    edd = lstn.distance(
                                        gtTrans[gtNum].upper(), detTrans[detNum].upper()
                                    )
                                    # if(resFile == '0001541'):
                                    #   print(gtTrans[gtNum].upper(), detTrans[detNum].upper())
                                    if edd <= 0:
                                        correct = True
                                        x += 1
                                        a += 1
                                    else:
                                        correct = False
                                        y += 1
                                        b += 1

                                    # print(detNum, " ", gtTrans[gtNum], " ", detTrans[detNum], " ", correct)
                                    # correct = gtTrans[gtNum].upper() == detTrans[detNum].upper()
                                else:
                                    try:
                                        correct = (
                                            transcription_match(
                                                gtTrans[gtNum].upper(),
                                                detTrans[detNum].upper(),
                                                evaluationParams["SPECIAL_CHARACTERS"],
                                                evaluationParams[
                                                    "ONLY_REMOVE_FIRST_LAST_CHARACTER"
                                                ],
                                            )
                                            == True
                                        )
                                    except:  # empty
                                        correct = False
                                detCorrect += 1 if correct else 0
                                if correct:
                                    detMatchedNums.append(detNum)

                for gtNum in range(len(gtPols)):
                    for detNum in range(len(detPols)):
                        if (
                            det_only_gtRectMat[gtNum] == 0
                            and det_only_detRectMat[detNum] == 0
                            and gtNum not in det_only_gtDontCarePolsNum
                            and detNum not in det_only_detDontCarePolsNum
                        ):
                            if (
                                iouMat[gtNum, detNum]
                                > evaluationParams["IOU_CONSTRAINT"]
                            ):
                                det_only_gtRectMat[gtNum] = 1
                                det_only_detRectMat[detNum] = 1
                                # detection matched only if transcription is equal
                                det_only_correct = True
                                detOnlyCorrect += 1

        numGtCare = len(gtPols) - len(gtDontCarePolsNum)
        numDetCare = len(detPols) - len(detDontCarePolsNum)
        # print("numDetCare: ", numDetCare)
        # print("len(detPols): ", len(detPols))
        # print("len(detDontCarePolsNum): ", len(detDontCarePolsNum))
        # print("a: ", a)
        # print("b: ", b)
        # print("a + b: ", a+b)
        # if(numGtCare != a + b):
          # print(resFile)
          # print(gtPols)
          # print(gtDontCarePolsNum)

        fail = (numDetCare - a - b)
        if (fail > max_fail and (resFile != '0001774' and resFile != '0001509' and resFile != '0001755'
                                 and resFile != '0001560'
                                 and resFile != '0001716'
                                 and resFile != '0001772'
                                 and resFile != '0001634'
                                 and resFile != '0001833'
                                 and resFile != '0001554'
                                  and resFile != '0001570'
                                and resFile != '0001713'
                                and resFile != '0001585'
                                and resFile != '0001508'
                                and resFile != '0001662'
                                and resFile != '0001738'
                                and resFile != '0001848'
                                and resFile != '0001541'
                                and resFile != '0001603'
                                and resFile != '0001836'
                                and resFile != '0001725'
                                and resFile != '0001676'
                                and resFile != '0001778'
                                and resFile != '0001640'
                                and resFile != '0001990'
                                and resFile != '0001995'
                                and resFile != '0001623'
                                and resFile != '0001751'
                                and resFile != '0001999'
                                and resFile != '0001517'
                                and resFile != '0001528'
                                and resFile != '0001531'
                                and resFile != '0001639')):
          max_fail = fail
          name_fail = resFile

        det_only_numGtCare = len(gtPols) - len(det_only_gtDontCarePolsNum)
        det_only_numDetCare = len(detPols) - len(det_only_detDontCarePolsNum)
        if numGtCare == 0:
            recall = float(1)
            precision = float(0) if numDetCare > 0 else float(1)
        else:
            recall = float(detCorrect) / numGtCare
            precision = 0 if numDetCare == 0 else float(detCorrect) / numDetCare

        if det_only_numGtCare == 0:
            det_only_recall = float(1)
            det_only_precision = float(0) if det_only_numDetCare > 0 else float(1)
        else:
            det_only_recall = float(detOnlyCorrect) / det_only_numGtCare
            det_only_precision = (
                0
                if det_only_numDetCare == 0
                else float(detOnlyCorrect) / det_only_numDetCare
            )

        hmean = (
            0
            if (precision + recall) == 0
            else 2.0 * precision * recall / (precision + recall)
        )
        det_only_hmean = (
            0
            if (det_only_precision + det_only_recall) == 0
            else 2.0
            * det_only_precision
            * det_only_recall
            / (det_only_precision + det_only_recall)
        )

        matchedSum += detCorrect
        det_only_matchedSum += detOnlyCorrect
        numGlobalCareGt += numGtCare
        numGlobalCareDet += numDetCare
        det_only_numGlobalCareGt += det_only_numGtCare
        det_only_numGlobalCareDet += det_only_numDetCare

        perSampleMetrics[resFile] = {
            "precision": precision,
            "recall": recall,
            "hmean": hmean,
            "iouMat": [] if len(detPols) > 100 else iouMat.tolist(),
            "gtPolPoints": gtPolPoints,
            "detPolPoints": detPolPoints,
            "gtTrans": gtTrans,
            "detTrans": detTrans,
            "gtDontCare": gtDontCarePolsNum,
            "detDontCare": detDontCarePolsNum,
            "evaluationParams": evaluationParams,
        }

    print("X: ", x)
    print("Y: ", y)
    print("X+Y: ", x+y)
    print("matchedSum: ", matchedSum)
    print("numGlobalCareDet: ", numGlobalCareDet)
    print("max_fail: ", max_fail)
    print("name_fail: ", name_fail)
    print("-------")
    print("numGlobalCareGt: ", numGlobalCareGt)
    methodRecall = 0 if numGlobalCareGt == 0 else float(matchedSum) / numGlobalCareGt
    methodPrecision = (
        0 if numGlobalCareDet == 0 else float(matchedSum) / numGlobalCareDet
    )
    methodHmean = (
        0
        if methodRecall + methodPrecision == 0
        else 2 * methodRecall * methodPrecision / (methodRecall + methodPrecision)
    )

    det_only_methodRecall = (
        0
        if det_only_numGlobalCareGt == 0
        else float(det_only_matchedSum) / det_only_numGlobalCareGt
    )
    det_only_methodPrecision = (
        0
        if det_only_numGlobalCareDet == 0
        else float(det_only_matchedSum) / det_only_numGlobalCareDet
    )
    det_only_methodHmean = (
        0
        if det_only_methodRecall + det_only_methodPrecision == 0
        else 2
        * det_only_methodRecall
        * det_only_methodPrecision
        / (det_only_methodRecall + det_only_methodPrecision)
    )

    methodMetrics = r"E2E_RESULTS: precision: {}, recall: {}, hmean: {}".format(
        methodPrecision, methodRecall, methodHmean
    )
    det_only_methodMetrics = (
        r"DETECTION_ONLY_RESULTS: precision: {}, recall: {}, hmean: {}".format(
            det_only_methodPrecision, det_only_methodRecall, det_only_methodHmean
        )
    )

    resDict = {
        "calculated": True,
        "Message": "",
        "e2e_method": methodMetrics,
        "det_only_method": det_only_methodMetrics,
        "per_sample": perSampleMetrics,
    }

    return resDict


def load_zip_file(file, fileNameRegExp="", allEntries=False):
    """
    Returns an array with the contents (filtered by fileNameRegExp) of a ZIP file.
    The key's are the names or the file or the capturing group definied in the fileNameRegExp
    allEntries validates that all entries in the ZIP file pass the fileNameRegExp
    """
    try:
        archive = zipfile.ZipFile(file, mode="r", allowZip64=True)
    except:
        raise Exception("Error loading the ZIP archive")

    pairs = []
    for name in archive.namelist():
        addFile = True
        keyName = name
        if fileNameRegExp != "":
            m = re.match(fileNameRegExp, name)
            if m == None:
                addFile = False
            else:
                if len(m.groups()) > 0:
                    keyName = m.group(1)

        if addFile:
            pairs.append([keyName, archive.read(name)])
        else:
            if allEntries:
                raise Exception("ZIP entry not valid: %s" % name)

    return dict(pairs)


def decode_utf8(raw):
    """
    Returns a Unicode object on success, or None on failure
    """
    try:
        raw = codecs.decode(raw, "utf-8", "replace")
        # extracts BOM if exists
        raw = raw.encode("utf8")
        if raw.startswith(codecs.BOM_UTF8):
            raw = raw.replace(codecs.BOM_UTF8, "", 1)
        return raw.decode("utf-8")
    except:
        return None


def get_tl_line_values_from_file_contents(
    content,
    CRLF=True,
    LTRB=True,
    withTranscription=False,
    withConfidence=False,
    imWidth=0,
    imHeight=0,
    sort_by_confidences=True,
):
    """
    Returns all points, confindences and transcriptions of a file in lists. Valid line formats:
    xmin,ymin,xmax,ymax,[confidence],[transcription]
    x1,y1,x2,y2,x3,y3,x4,y4,[confidence],[transcription]
    """
    pointsList = []
    transcriptionsList = []
    confidencesList = []

    lines = content.split("\r\n" if CRLF else "\n")
    for line in lines:
        line = line.replace("\r", "").replace("\n", "")
        if line != "":
            points, confidence, transcription = get_tl_line_values_gt(
                line, LTRB, withTranscription, withConfidence, imWidth, imHeight
            )
            pointsList.append(points)
            transcriptionsList.append(transcription)
            confidencesList.append(confidence)

    # print("NOTE 4: ", transcriptionsList)
    # print("len(tran)", len(transcriptionsList))
    if withConfidence and len(confidencesList) > 0 and sort_by_confidences:
        import numpy as np

        sorted_ind = np.argsort(-np.array(confidencesList))
        confidencesList = [confidencesList[i] for i in sorted_ind]
        pointsList = [pointsList[i] for i in sorted_ind]
        transcriptionsList = [transcriptionsList[i] for i in sorted_ind]

    # print("NOTE 5: ", transcriptionsList)
    return pointsList, confidencesList, transcriptionsList


def get_tl_line_values_from_file_contents_det(
    content,
    CRLF=True,
    LTRB=True,
    withTranscription=False,
    withConfidence=False,
    imWidth=0,
    imHeight=0,
    sort_by_confidences=True,
):
    """
    Returns all points, confindences and transcriptions of a file in lists. Valid line formats:
    xmin,ymin,xmax,ymax,[confidence],[transcription]
    x1,y1,x2,y2,x3,y3,x4,y4,[confidence],[transcription]
    """
    pointsList = []
    transcriptionsList = []
    confidencesList = []

    lines = content.split("\r\n" if CRLF else "\n")
    for line in lines:
        line = line.replace("\r", "").replace("\n", "")
        if line != "":
            points, confidence, transcription = get_tl_line_values(
                line, LTRB, withTranscription, withConfidence, imWidth, imHeight
            )
            pointsList.append(points)
            transcriptionsList.append(transcription)
            confidencesList.append(confidence)

    if withConfidence and len(confidencesList) > 0 and sort_by_confidences:
        import numpy as np

        sorted_ind = np.argsort(-np.array(confidencesList))
        confidencesList = [confidencesList[i] for i in sorted_ind]
        pointsList = [pointsList[i] for i in sorted_ind]
        transcriptionsList = [transcriptionsList[i] for i in sorted_ind]

    return pointsList, confidencesList, transcriptionsList


def get_tl_line_values_gt(
    line,
    LTRB=True,
    withTranscription=False,
    withConfidence=False,
    imWidth=0,
    imHeight=0,
):
    """
    Validate the format of the line. If the line is not valid an exception will be raised.
    If maxWidth and maxHeight are specified, all points must be inside the imgage bounds.
    Posible values are:
    LTRB=True: xmin,ymin,xmax,ymax[,confidence][,transcription]
    LTRB=False: x1,y1,x2,y2,x3,y3,x4,y4[,confidence][,transcription]
    Returns values from a textline. Points , [Confidences], [Transcriptions]
    """
    confidence = 0.0
    transcription = ""
    points = []

    if LTRB:
        # do not use
        raise Exception("Not implemented.")

    else:
        # if withTranscription and withConfidence:
        #     cors = line.split(',')
        #     assert(len(cors)%2 -2 == 0), 'num cors should be even.'
        #     try:
        #         points = [ float(ic) for ic in cors[:-2]]
        #     except Exception as e:
        #         raise(e)
        # elif withConfidence:
        #     cors = line.split(',')
        #     assert(len(cors)%2 -1 == 0), 'num cors should be even.'
        #     try:
        #         points = [ float(ic) for ic in cors[:-1]]
        #     except Exception as e:
        #         raise(e)
        # elif withTranscription:
        #     cors = line.split(',')
        #     assert(len(cors)%2 -1 == 0), 'num cors should be even.'
        #     try:
        #         points = [ float(ic) for ic in cors[:-1]]
        #     except Exception as e:
        #         raise(e)
        # else:
        #     cors = line.split(',')
        #     assert(len(cors)%2 == 0), 'num cors should be even.'
        #     try:
        #         points = [ float(ic) for ic in cors[:]]
        #     except Exception as e:
        #         raise(e)

        if withTranscription and withConfidence:
            raise ("not implemented")
        elif withConfidence:
            raise ("not implemented")
        elif withTranscription:
            ptr = line.strip().split(",####")
            # print("line ", line)
            cors = ptr[0].split(",")
            recs = ptr[1].strip()
            assert len(cors) % 2 == 0, "num cors should be even."
            try:
                points = [float(ic) for ic in cors[:]]
            except Exception as e:
                raise (e)
        else:
            raise ("not implemented")

        #        validate_clockwise_points(points)

        if imWidth > 0 and imHeight > 0:
            for ip in range(0, len(points), 2):
                validate_point_inside_bounds(
                    points[ip], points[ip + 1], imWidth, imHeight
                )

    if withConfidence:
        try:
            confidence = 1.0
        except ValueError:
            raise Exception("Confidence value must be a float")

    if withTranscription:
        # posTranscription = numPoints + (2 if withConfidence else 1)
        # transcription = cors[-1].strip()
        transcription = recs
        m2 = re.match(r"^\s*\"(.*)\"\s*$", transcription)
        if (
            m2 != None
        ):  # Transcription with double quotes, we extract the value and replace escaped characters
            transcription = m2.group(1).replace("\\\\", "\\").replace('\\"', '"')

    return points, confidence, transcription


def get_tl_line_values(
    line,
    LTRB=True,
    withTranscription=False,
    withConfidence=False,
    imWidth=0,
    imHeight=0,
):
    """
    Validate the format of the line. If the line is not valid an exception will be raised.
    If maxWidth and maxHeight are specified, all points must be inside the imgage bounds.
    Posible values are:
    LTRB=True: xmin,ymin,xmax,ymax[,confidence][,transcription]
    LTRB=False: x1,y1,x2,y2,x3,y3,x4,y4[,confidence][,transcription]
    Returns values from a textline. Points , [Confidences], [Transcriptions]
    """
    confidence = 0.0
    transcription = ""
    points = []

    # print("withConfidence: ", withConfidence)

    if LTRB:
        # do not use
        raise Exception("Not implemented.")

    else:
        if withTranscription and withConfidence:
            raise ("not implemented")
        elif withConfidence:
            raise ("not implemented")
        elif withTranscription:
            ptr = line.strip().split(",####")
            cors = ptr[0].split(",")
            recs = ptr[1].strip()
            assert len(cors) % 2 == 0, "num cors should be even."
            try:
                points = [float(ic) for ic in cors[:]]
            except Exception as e:
                raise (e)
        else:
            raise ("not implemented")

        # print('det clock wise')
        #        validate_clockwise_points(points)

        if imWidth > 0 and imHeight > 0:
            for ip in range(0, len(points), 2):
                validate_point_inside_bounds(
                    points[ip], points[ip + 1], imWidth, imHeight
                )

    if withConfidence:
        try:
            confidence = 1.0
        except ValueError:
            raise Exception("Confidence value must be a float")

    if withTranscription:
        # posTranscription = numPoints + (2 if withConfidence else 1)
        transcription = recs
        # print("NOTE 1: ", transcription)
        m2 = re.match(r"^\s*\"(.*)\"\s*$", transcription)
        if (
            m2 != None
        ):  # Transcription with double quotes, we extract the value and replace escaped characters
            # print("NOTE 2: ", transcription)
            transcription = m2.group(1).replace("\\\\", "\\").replace('\\"', '"')

    # print("NOTE 1: ", transcription)
    return points, confidence, transcription


def validate_point_inside_bounds(x, y, imWidth, imHeight):
    if x < 0 or x > imWidth:
        raise Exception(
            "X value (%s) not valid. Image dimensions: (%s,%s)" % (x, imWidth, imHeight)
        )
    if y < 0 or y > imHeight:
        raise Exception(
            "Y value (%s)  not valid. Image dimensions: (%s,%s) Sample: %s Line:%s"
            % (y, imWidth, imHeight)
        )


dir_to_val = "./"+ destination_dir + ".zip"
evaluate_with_official_code(dir_to_val, "./vintext_label.zip");
# evaluate_with_official_code("./vintext_label.zip", "./vintext_label.zip");
print(dir_to_val)

submission file path:  ./res_output_filtered_thrs_0.0.zip
gt file path:  ./vintext_label.zip
X:  6554
Y:  1784
X+Y:  8338
matchedSum:  6554
numGlobalCareDet:  12557
max_fail:  7
name_fail:  0001501
-------
numGlobalCareGt:  9769
Calculated!
"E2E_RESULTS: precision: 0.5219399538106235, recall: 0.6708977377418365, hmean: 0.5871181582012003"
"DETECTION_ONLY_RESULTS: precision: 0.6635427155139029, recall: 0.8493952012690859, hmean: 0.7450537026568683"
./res_output_filtered_thrs_0.0.zip
